In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Input
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
%matplotlib inline
import helper  # courtesy of CS109B

## Image Classification: FFNN vs CNN

In this exercise we will train a neural network to tell these two images apart.

Pavlos          |  Not Pavlos
:-------------------------:|:-------------------------:
![title](data/pavlos.jpeg) |![title](data/not-pavlos.jpeg)

Surely this is a simple task with only two images! But there is a catch. We will use an image generator to create 'translated' versions of our two images. That is, images shifted up or down, left or right. In this way every image the network sees with me a novel variation.

### Image Generator

This generator will provide our NNs with randomly translated versions of the above images.

In [2]:
img_generator = helper.get_generator()

print(f'Our classes: {img_generator.class_indices}')

TARGET_SIZE = img_generator.target_size
print(f'Generator produces images of size {TARGET_SIZE} (with 3 color channels)')

BATCH_SIZE = img_generator.batch_size
print(f'Images are generated in batches of size {BATCH_SIZE}')

FileNotFoundError: [Errno 2] No such file or directory: 'data/Ed/train'

In [ ]:
sample_batch = img_generator.next()[0]
fig, ax = plt.subplots(4,4)
ax = ax.ravel()
for i, img in enumerate(sample_batch):
    ax[i].set_axis_off()
    ax[i].imshow(img)
plt.suptitle('Sample Batch of Generated Images', y=1.05)
plt.tight_layout()

### Feed-Forward Neural Network

Our first network will be a feed-forward neural network. The only layers with learned parameters we will be using are dense layers.

In [ ]:
FFNN = Sequential()
# compare input_shape to TARGET_SIZE above
FFNN.add(Input(shape=(150, 150, 3)))
# fill in the layer needed at the beginning of our FFNN for it to process images
# Ex: FFNN.add(Somelayer())  
# Hint: check the imports above
FFNN.add(__)  
# specify a list of the number of nodes for each dense layer
# you can try any number of dense layers with any number of nodes in each
# Ex: for n_nodes in [a,b,c,..] where x,y,z, etc. are ints
for n_nodes in [____]:
    FFNN.add(Dense(n_nodes, activation='relu'))
FFNN.add(Dense(1, activation='sigmoid'))

FFNN.compile(loss='binary_crossentropy', metrics=['accuracy'])

How do your node and layer number choices affect the number of learned parameters?

Can your FFNN do better than chance guessing after 10 epochs?

For a real challenge, see if you can do it with fewer than 5 million parameters.

In [ ]:
FFNN.summary()

In [ ]:
FFNN_history = FFNN.fit(
        img_generator,
        steps_per_epoch=300// BATCH_SIZE,
        epochs=10,
        validation_data=img_generator,
        validation_steps=75// BATCH_SIZE)

In [ ]:
def plot_history(history, name):
    fig, ax = plt.subplots(1,2, figsize=(8,3))
    for i, metric in enumerate(['loss', 'accuracy']):
        ax[i].plot(history.history[metric], label='train')
        ax[i].plot(history.history[f'val_{metric}'], label='val')
        if metric == 'accuracy': ax[i].axhline(0.5, c='r', ls='--', label='trivial accuracy')
        ax[i].set_xlabel('epoch')
        ax[i].set_ylabel(metric)
    plt.suptitle(f'{name} Training', y=1.05)
    plt.legend()
    plt.tight_layout()

plot_history(FFNN_history, 'Feed-Forward Neural Network')

In [ ]:
_, FFNN_acc = FFNN.evaluate(img_generator, steps=15)
print(f'FFNN Test Accuracy: {FFNN_acc}')

### CNN

The CNN offers two great advantages over the FFNN in this task:
1. Far Fewer Parameters

The FFNN had weights between _every_ input pixel and each node in the first dense layer. That's a lot of weights! By contrast, the weights learned by the first CNN layer are not a function of the size of the input image at all. The depend only on the size and number of filters.
2. Learning Translation Invariant Features

Features are detected by the filters which convolve across the entire image. This means then can recognize the feature they are tuned to no matter where it occurs in an image. The FFNN has no way of representing 'translated' features as being the same. It must learn each position independantly!

In [ ]:
CNN = Sequential()
CNN.add(Input(shape=(150, 150, 3)))
# specify a list of the number of filters for each convolutional layer
# you can try any number of convolutional layers with any number of filters in each
# Ex: for n_filters in [a,b,c,..] where a,b,c,etc. are ints
for n_filters in [____]:
    CNN.add(Conv2D(n_filters, kernel_size=3, activation='relu'))
    # add a layer to further reduce the dimensionality
    # Hint: this layer has no learned parameters of its own
    CNN.add(__)
# fill in the layer needed between our 2d convolutional layers and the dense layer
CNN.add(__)
# specify the number of nodes in the dense layer before the output
CNN.add(Dense(__, activation='relu'))
CNN.add(Dense(1, activation='sigmoid'))
    
CNN.compile(loss='binary_crossentropy', metrics=['accuracy'])

How do your choices above affect the number of parameters?

Work to get above 95% accuracy after 10 epochs with 100k parameters or fewer.

In [ ]:
CNN.summary()

In [ ]:
CNN_history = CNN.fit(
        img_generator,
        steps_per_epoch=300 // BATCH_SIZE,
        epochs=10,
        validation_data=img_generator,
        validation_steps=75// BATCH_SIZE)

In [ ]:
plot_history(CNN_history, 'Convolutional Neural Network')

In [ ]:
_, CNN_acc = CNN.evaluate(img_generator, steps=15)
print(f'CNN Test Accuracy: {CNN_acc}')